Import libraries

In [ ]:
import pandas as pd
import boto3
import json
import configparser

Read the Data WareHouse configuration details to create a Redshift Cluster

Create clients for EC2, S3, IAM, and RedShift

Create a local Spark session